# PW4: Genetic algorithms for optimization
*Alexandra Korukova, Samuel Mayor*  
*HEIG-VD - MLG - 30.05.2019*

In [44]:
from scipy import sin, cos, tan, arctan, arctan2, arccos, pi
import numpy as np
import math
from pyevolve import G1DList
from pyevolve import GSimpleGA
from pyevolve import GAllele
from pyevolve import Mutators
from pyevolve import Initializators
from pyevolve import DBAdapters
from pyevolve import Crossovers
from pyevolve import Consts
import sys, random

In [45]:
LAT = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47, 17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]

LON = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02, 96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]

In [46]:
def tour_length(matrix, tour):
   """ Returns the total length of the tour """
   total=0
   num_cities=len(tour)
   for i in range(num_cities):
      j=(i+1)%num_cities
      city_i=tour[i]
      city_j=tour[j]
      total+=matrix[city_i,city_j]
   return total

def G1DListTSPInitializator(genome, **args):
   """ The initializator for the TSP """
   genome.clearList()
   lst = [i for i in xrange(genome.getListSize())]

   for i in xrange(genome.getListSize()):
      choice = random.choice(lst)
      lst.remove(choice)
      genome.append(choice)

spherical_distances = []
coords = []

def eval_func(chromosome):
   """ The evaluation function """
   global spherical_distances
   return tour_length(spherical_distances, chromosome)

In [51]:
# source : https://www.johndcook.com/blog/python_longitude_latitude/
def spherical_distance(lat1, long1, lat2, long2):

    # Convert latitude and longitude to
    # spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0

    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians

    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians

    # Compute spherical distance from spherical coordinates.

    # For two locations in spherical coordinates
    # (1, theta, phi) and (1, theta', phi')
    # cosine( arc length ) =
    # sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length

    cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) +
    math.cos(phi1)*math.cos(phi2))
    arc = math.acos( cos )

    # Remember to multiply arc by the radius of the earth
    # in your favorite set of units to get length.
    return arc*6371

In [55]:
# creates latitude/longitude tuples for each city
coords = []
for i in range(len(LAT)):
    coords.append((LAT[i], LON[i]))

# calculate distances between each city pairs
spherical_distances = np.zeros((14, 14))
for i,c1 in enumerate(coords):
    for j,c2 in enumerate(coords):
        spherical_distances[i, j] = spherical_distance(c1[0], c1[1], c2[0], c2[1])

# set the alleles to the cities numbers
setOfAlleles = GAllele.GAlleles(homogeneous=True)
lst = [ i for i in xrange(len(coords)) ]
a = GAllele.GAlleleList(lst)
setOfAlleles.add(a)

genome = G1DList.G1DList(len(coords))
genome.setParams(allele=setOfAlleles)

genome.evaluator.set(eval_func)
genome.mutator.set(Mutators.G1DListMutatorSwap)
genome.crossover.set(Crossovers.G1DListCrossoverOX)
genome.initializator.set(G1DListTSPInitializator)

for i in np.arrange(100, 1001, 100):
    ga = GSimpleGA.GSimpleGA(genome)
    ga.setGenerations(i)
    ga.setMinimax(Consts.minimaxType["minimize"])
    ga.setCrossoverRate(1.0)
    ga.setMutationRate(0.03)
    ga.setPopulationSize(80)

    ga.evolve(freq_stats=100)
    best = ga.bestIndividual()
    print best
    print(tour_length(spherical_distances, best))


Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw) [8118.14(8290.22)/5318.98(5135.05)/6765.11(6765.11)]
Gen. 100 (10.00%): Max/Min/Avg Fitness(Raw) [5416.20(6125.86)/4370.82(4258.66)/4513.50(4513.50)]
Gen. 200 (20.00%): Max/Min/Avg Fitness(Raw) [5280.12(5919.38)/4243.96(4130.53)/4400.10(4400.10)]
Gen. 300 (30.00%): Max/Min/Avg Fitness(Raw) [4759.95(6028.10)/3818.13(3580.75)/3966.63(3966.63)]
Gen. 400 (40.00%): Max/Min/Avg Fitness(Raw) [4490.50(5880.57)/3619.63(3392.19)/3742.08(3742.08)]
Gen. 500 (50.00%): Max/Min/Avg Fitness(Raw) [4307.83(6085.22)/3532.99(3392.19)/3589.86(3589.86)]
Gen. 600 (60.00%): Max/Min/Avg Fitness(Raw) [4327.23(5965.76)/3540.67(3392.19)/3606.02(3606.02)]
Gen. 700 (70.00%): Max/Min/Avg Fitness(Raw) [4498.68(6396.77)/3647.89(3392.19)/3748.90(3748.90)]
Gen. 800 (80.00%): Max/Min/Avg Fitness(Raw) [4430.42(5584.10)/3575.00(3392.19)/3692.01(3692.01)]
Gen. 900 (90.00%): Max/Min/Avg Fitness(Raw) [4713.78(7040.13)/3792.84(3392.19)/3928.15(3928.15)]
Gen. 1000 (100.00%): Max/Min/Avg 

In [54]:
print(spherical_distances)

[[0.00000000e+00 1.77009305e+02 5.50675649e+02 7.17757240e+02
  9.81224413e+02 6.14930327e+02 4.55234726e+02 8.36531369e+01
  1.37851129e+02 3.45482052e+02 1.36631089e+02 5.64086509e+02
  3.44588856e+02 4.34508524e+02]
 [1.77009305e+02 0.00000000e+00 4.49724987e+02 6.67756396e+02
  1.01647667e+03 6.37693019e+02 5.21372205e+02 2.12968085e+02
  3.14202703e+02 4.77725171e+02 3.13518919e+02 5.74393117e+02
  4.33391525e+02 4.02693079e+02]
 [5.50675649e+02 4.49724987e+02 0.00000000e+00 2.69825166e+02
  7.47657237e+02 4.21626921e+02 4.69163853e+02 5.09234088e+02
  6.62466452e+02 8.95787809e+02 6.46241876e+02 3.54657083e+02
  4.86258069e+02 2.09901390e+02]
 [7.17757240e+02 6.67756396e+02 2.69825166e+02 0.00000000e+00
  5.04643144e+02 2.79304235e+02 4.33921938e+02 6.52907115e+02
  7.97079289e+02 1.05408788e+03 7.75318270e+02 2.48544627e+02
  5.12152028e+02 2.83479792e+02]
 [9.81224413e+02 1.01647667e+03 7.47657237e+02 5.04643144e+02
  9.49352980e-05 3.79063065e+02 5.29767283e+02 8.98296394e+02


In [35]:
print(ga)

- GSimpleGA
	GP Mode:		 False
	Population Size:	 80
	Generations:		 1000
	Current Generation:	 1000
	Mutation Rate:		 0.03
	Crossover Rate:		 1.00
	Minimax Type:		 Minimize
	Elitism:		 True
	Elitism Replacement:	 1
	DB Adapter:		 None
	Slot [Selector] (Count: 1)
		Name: GRankSelector - Weight: 0.50
		Doc:  The Rank Selector - This selector will pick the best individual of
   the population every time.
   
	Slot [Generation Step Callback] (Count: 0)
		No function
	Slot [Termination Criteria] (Count: 0)
		No function


